In [80]:
import string

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import _stop_words
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import feature_extraction, pipeline, linear_model, metrics

from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np

from geotext import GeoText

In [81]:
def remove_pun(text):
    for pun in string.punctuation:
        text = text.replace(pun, "")
    text = text.lower()
    return text

In [143]:
df = pd.read_csv("Sentences.csv")

# df["sentence"] = df["sentence"].apply(remove_pun)

In [155]:
# Define a function to extract cities from a sentence
def extract_cities_from_sentence(sentence):
    places = GeoText(sentence)
    cities = list(places.cities)
    if len(cities) >= 2:
        return cities[0], cities[1]
    elif len(cities) == 1:
        return cities[0], None
    else:
        return None, None

# Apply the extract_cities_from_sentence function to each row in the DataFrame
df[["city_1", "city_2"]] = df["sentence"].apply(lambda x: pd.Series(extract_cities_from_sentence(x)))

all_sentences = [f"{sentence}, {city1}, {city2}" for sentence, city1, city2 in df[["sentence", "city_1", "city_2"]].values]
input_values = [sentence.lower().replace(r'[^\w\s]', '') for sentence in all_sentences]

# input_values = df["sentence"]

In [156]:
stemmer = SnowballStemmer('french')
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

vectorizer = CountVectorizer(stop_words='french', analyzer=stemmed_words)
X = vectorizer.fit_transform(input_values)
y = df.iloc[:, 1] + " to " + df.iloc[:, 2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [157]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 7.75%


In [158]:
df_pred = df[-y_pred.size:]
df_pred.loc[df_pred.index, "prediction"] = y_pred

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [115]:
def check_cities_in_sentence(row):
    return row['prediction'].split(" to ")[0] in row['sentence'] and row['prediction'].split(" to ")[0] in row['sentence']

# Apply the function to create a new column 'city_match' that stores the boolean result
df_pred['city_match'] = df_pred.apply(check_cities_in_sentence, axis=1)

# Now, filter the rows where 'city_match' is True
filtered_df = df_pred[df_pred['city_match']]

# Drop the 'city_match' column if you don't need it
filtered_df = filtered_df.drop(columns=['city_match'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [118]:
df_pred

,sentence,departure,arrival,city_1,city_2,prediction,city_match
1600,Je veux prendre le train de Clermont-Ferrand à...,Clermont-Ferrand,Saint-Denis,Clermont-Ferrand,Saint-Denis,Nantes to Saint-Denis,False
1601,Le train de Paris à Rouen offre une vue magnif...,Paris,Rouen,Paris,Rouen,Nice to Le Mans,False
1602,Je prévois de voyager de Rouen à Marseille en ...,Rouen,Marseille,Rouen,Marseille,Marseille to Saint-Étienne,True
1603,"Pour mes prochaines vacances, je vais de Bézie...",Béziers,Tourcoing,Béziers,Tourcoing,Le Mans to Rouen,False
1604,Je souhaite découvrir Calais en prenant le tra...,Nîmes,Calais,Calais,Nîmes,Tourcoing to Nîmes,False
...,...,...,...,...,...,...,...
1995,Je veux prendre le train de Bordeaux à La Roch...,Bordeaux,La Rochelle,Bordeaux,La Rochelle,Bordeaux to Limoges,True
1996,Le train est le meilleur moyen de se rendre de...,Nice,Lille,Nice,Lille,Tourcoing to Nîmes,False
1997,"Allons de Pau à Toulon en train, qu'en penses-...",Pau,Toulon,Toulon,None,Amiens to Toulon,False
1998,Je souhaite découvrir Marseille en prenant le ...,Orléans,Marseille,Marseille,Orléans,Toulon to Lille,False


In [138]:
for index, row in df.head().iterrows():
    print(row["sentence"])
    
#     X = vectorizer.fit_transform([row["sentence"]])
    
#     prediction = clf.predict(X)
    
#     print(prediction)

Il est temps de visiter Rouen. Je vais prendre le train depuis Nîmes.
La gare de Rouen à Béziers est-elle bien desservie ?
Allons de Aix-en-Provence à Lille en train, qu'en penses-tu ?
Nantes est une belle ville. Je veux y aller en train depuis Clermont-Ferrand.
J'ai prévu de partir en train de Nantes à Aix-en-Provence.
